# <center> <font color = 'rebeccapurple'>Stochastic Watershed Model: Generating Stochastic Streamflows for the Shasta River Under 2 Degrees of Warming</font> </center>

This Jupyter Notebook contains code for a stochastic watershed model (SWM) that can be used to develop an ensemble of stochastic streamflows for a single site. This code was used to produce the results found in **Shabestanipour et al. (2024)** for the Shasta River basin. With modifications to the input data and the specific ARMA model used (more details below), this code could be applied to other non-regulated river basins. 

This notebook is split into two parts: 1. SWM (generating stochastic streamflows) and 2. Diagnostics (assessing stochastic model performance).

***

**Computational Requirements**: We used Jupyter version 4.7.1 and Python version 3.8.8 to run the below code. Lower versions may result in error messages.

The code is somewhat computationally-intesnive. We found that allocating 32GB of memory allowed the code to run in full without crashing the kernel. 

The code blocks that will take the longest are generating the stochastic residuals in Part 1 and generating the figure inputs in Part 2 (depending on how many stochastic realizations you do). These should each take approx. 10 min maximum. Most other code blocks run very quickly. The code to generate AIC/BIC for various ARIMA models to determine which to use may also take some time (by default this code block is commented out).

***

# <center> <font color = 'red'> Part 1: SWM </font> </center>

## <font color='green'> Import Statements </font>

Here we import SWM functions (developed by Ghazal Shabestanipour) as well as existing external Python packages used in the notebook. Be sure that the file **Func_Lib.py** is contained in the same directory as this Jupyter Notebook.
For existing Python packages, if you do not have a particular package installed with your version of Python, you can install it using pip or conda (if using anaconda). Instructions for doing so can be found here:<br>
https://packaging.python.org/en/latest/tutorials/installing-packages/ (pip)<br>
https://docs.conda.io/projects/conda/en/latest/user-guide/concepts/installing-with-conda.html (conda)

In [1]:
# functions developed by Ghazal:
import Func_Lib_forAbby as func

# general packages
import numpy as np #numpy is a scientific computing package
import pandas as pd #pandas is a data analysis package
import math #math contains math functions
import time #functions in this package can be used to understand how long the code is taking to run
import os #functions related to operating system

# plotting packages
import matplotlib.pyplot as plt #matplotlib.pyplot is used for plotting
import seaborn as sns #seaborn is a plotting package
plt.style.use('seaborn-white') #this is the specific style of plotting used

# statistical analysis packages
from scipy import stats #scipy.stats are functions related to statistical analysis
from scipy.stats import norm #functions for normally distributed random variable 

# time series analysis packages
import statsmodels as sm
from statsmodels.graphics.tsaplots import plot_acf #function to plot autocorrelation function (acf) for time series analysis
from statsmodels.graphics.tsaplots import plot_pacf #function to plot partial autocorrelation function (pacf) for time series analysis
#from statsmodels.tsa.arima_model import ARIMA #import function for ARIMA (time series analysis)
from statsmodels.tsa.arima.model import ARIMA

## <font color='green'> Load Input Data </font>

The only input data required is a csv containing daily streamflow data (observed and modeled). The columns of the csv are $date$, observed streamflow $Qgage$, deterministic model of streamflow $Qmodel$, and the difference between Qgage and Qmodel for each date $diff$.

For the Squannacook, we have values from 10/01/1987 - 09/30/2018.

In [2]:
# read in the csv of streamflow data and assign to variable called "data"
data = pd.read_csv('SWM-data_cond-debias.csv') #columns are date, Qgage (observed), Qmodel (deterministic model), and diff (Qmodel - Qgage)

data['month'] = pd.DatetimeIndex(data['date']).month #create a DatetimeIndex column for month
data['date'] = pd.to_datetime(data['date']) #convert format of "date" column to Datetime (for time series analysis)

#convert units to m^3/s 
# mm/day * SHA area in mi2 * (1609.34^2 m^2/mi^2) * (10^-3 m/1 mm) * (1 day/24*3600 s)
area_mi2_SHA = 6665 #area for Shasta baisn in sq. mi. from Zach Brodeur
data['Qgage'] = data['Qgage']*area_mi2_SHA*(1609.34**2)*(10**-3)/(24*3600)
data['Qmodel'] = data['Qmodel']*area_mi2_SHA*(1609.34**2)*(10**-3)/(24*3600)

# generating empirical log-ratio errors
data['lambda'] = np.log(data['Qmodel'] / data['Qgage']) #calculate the lambda = log(Qmodel/Qgage)
data['raw diff'] = data['Qmodel'] - data['Qgage']
data #display first five rows of data

,date,Qgage,Qmodel,month,lambda,raw diff
0,1987-10-01,59.938142,366.261850,10,1.810035,306.323708
1,1987-10-02,99.896904,735.416652,10,1.996299,635.519749
2,1987-10-03,99.896904,550.188717,10,1.706123,450.291814
3,1987-10-04,79.917523,421.519228,10,1.662870,341.601705
4,1987-10-05,99.896904,336.630618,10,1.214848,236.733714
...,...,...,...,...,...,...
11318,2018-09-26,79.917523,78.229589,9,-0.021347,-1.687934
11319,2018-09-27,79.917523,77.855393,9,-0.026142,-2.062129
11320,2018-09-28,59.938142,77.513991,9,0.257145,17.575849
11321,2018-09-29,79.917523,81.235286,9,0.016355,1.317763


In [3]:
#load in a "future" scenario into a new datafrae df_fut
df_fut = pd.read_csv('SWM-data_cond-debias_scenario-12.csv') #make sure this is one of the five we want, I just used a random one

#put the future scenario into the correct units of m3/s
df_fut['Qfut'] = df_fut['Qsim_cbias']*area_mi2_SHA*(1609.34**2)*(10**-3)/(24*3600)
df_fut['date'] = pd.to_datetime(df_fut['date'])
df_fut = df_fut[(df_fut['date'] > '1987-09-30') & (df_fut['date'] < '2018-10-01')]
df_fut = df_fut.reset_index()
df_fut['index']=np.arange(0, 11323)

#df_fut = df_fut.iloc[0:11323]

df_fut

,index,date,Qsim,Qsim_cbias,Qfut
0,0,1987-10-01,0.008558,0.415945,83.103222
1,1,1987-10-02,0.008422,0.415334,82.981132
2,2,1987-10-03,0.008271,0.414649,82.844289
3,3,1987-10-04,0.008132,0.414021,82.718931
4,4,1987-10-05,0.007975,0.413306,82.575901
...,...,...,...,...,...
11318,11318,2018-09-26,0.012123,0.384383,76.797403
11319,11319,2018-09-27,0.011791,0.382832,76.487560
11320,11320,2018-09-28,0.011474,0.381337,76.188862
11321,11321,2018-09-29,0.014408,0.394723,78.863275


## <font color='green'> Fit ARMA Model to Data </font>

Here we fit an AR4 model to our "lambdas" to get the model residuals and parameters.

In [4]:
p=4 #order of AR (autocorrelation with lag 4)
d=0 #order of I (no trend)
q=0 #order of MA (no moving average component)

mod = ARIMA(data['lambda'], order=(p,d,q)) #set up AR4 model using lambdas as input
res = mod.fit() #fit model

In [5]:
res.params #display the model parameters

const     0.023440
ar.L1     0.165057
ar.L2     0.187447
ar.L3     0.147261
ar.L4     0.137038
sigma2    0.067909
dtype: float64

Since we fit an AR4 model for this data, we have five parameters: a coefficient for each AR component (L1, L2, L3, and L4) and a constant term.

In [6]:
#create a column in data for the model residuals associated with each time step
data['Ar_res']=res.resid
data.head(5)

,date,Qgage,Qmodel,month,lambda,raw diff,Ar_res
0,1987-10-01,59.938142,366.261850,10,1.810035,306.323708,1.786596
1,1987-10-02,99.896904,735.416652,10,1.996299,635.519749,1.411201
2,1987-10-03,99.896904,550.188717,10,1.706123,450.291814,0.761985
3,1987-10-04,79.917523,421.519228,10,1.662870,341.601705,0.583921
4,1987-10-05,99.896904,336.630618,10,1.214848,236.733714,0.070037


## <font color = 'green'> Generate Random AR Parameter Sets </font>

We generate $m$ random AR parameter sets (called "Beta") to consider the uncertainty in the AR model parameters.

We use the Numpy function **random.multivariate_normal** to generate the parameter sets. This function samples the parameters from a multivariate normal distribution. The inputs to the function are the mean **(res.params)**, the covariance **(res.cov_params)**, and the number of parameter sets to generate. 

In [7]:
# m is the number of random AR parameter sets, Beta, generated for considering AR model uncertainty.
m = 100

# older version of ARIMA function
#Beta=np.random.multivariate_normal(np.array(res.params), np.array(res.cov_params()), m)

# updated Beta for using newer version of statsmodels ARIMA code
# Beta=np.random.multivariate_normal(np.array(res.params[:-1]), np.array(res.cov_params())[:-1,:-1], m)

# for reproducible Betas, use scipy.stats multivariate_normal function instead, setting random seed
Beta = stats.multivariate_normal(np.array(res.params[:-1]),np.array(res.cov_params())[:-1,:-1], seed=16)
Beta = Beta.rvs(m)

## <font color = 'green'> Generate Stochastic Residuals Using Simple Bootstrap </font>

There are 3 steps to generating the stochastic streamflow realizations:<br>

1. Generate stochastic residuals (use either simple bootstrap, conditional bootstrap, or k-Nearest Neighbors bootstrap)
2. Generate stochastic log errors by combining AR model uncertainty + stochastic residuals in AR4 model
3. Generate stochastic streamflow by transforming from log-ratios back to streamflows, including bias correction factor<br>

### <font color = 'green'> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Step 1: Generate Stochastic Residuals <green>

In [8]:
#now find the nearest Qmodel and residual associated with Qmodel for each flow value in Qfut
df_fut['Qmodel_nearest'] = 0 #initialize empty column for nearest Qmodel
df_fut['residual_nearest'] = 0 #initialize empty column for nearest Qmodel residual

for i in range(len(df_fut)): #loop through each row of df_fut
    #for each flow, calculate absolute difference of that flow and all flows in Qmodel
    #take absolute value and then find the index of the minimum absolute value
    #populate Qmodel_nearest column with data['Qmodel'] at this index
    #and residual_nearest column with data['Ar_res'] at this index
    df_fut['Qmodel_nearest'].iloc[i] = data['Qmodel'].iloc[abs(data['Qmodel'] - df_fut['Qfut'].iloc[i]).idxmin()]
    df_fut['residual_nearest'].iloc[i] = data['Ar_res'].iloc[abs(data['Qmodel'] - df_fut['Qfut'].iloc[i]).idxmin()]
    
#now use KNN bootstrapping as before, with Epsilon defined as the Qmodel_nearest and residual_nearest columns from df_fut
#basically exactly what we had before, with some data "repeats" to correspond to the new data points
#and in the correct order to match Qfut

n = 100 # n is the number of stochastic streamflow realizations generated for each Beta set
Epsilon = df_fut.filter(['residual_nearest','Qmodel_nearest'])
Epsilon = np.array(Epsilon) # turn dataframe into numpy array

E = [] #initialize empty list to store errors
for t in range(0,len(df_fut)): # loop through all time periods
    E.append(Epsilon - [0,Epsilon[t,1]])  # first column residual, second column diff between flow and all other flows

k = 750

Er = np.zeros((len(df_fut),k)) #initialize empty 2d array with dimensions time periods x k (# nearest neighbors)
for t in range(0,len(df_fut)): #loop through all time periods
    E[t][:,1] = np.abs(E[t][:,1]) #get absolute values of errors
    x = E[t][np.argsort(E[t][:, 1])] #sort errors by magnitude
    Er[t,:] = x[0:k,0]
    

R = np.zeros((len(df_fut),n*m)) #initialize empty 2d array with dimensions time periods x # realizations
for i in range(0,n*m): #loop through all realizations
    I = np.random.randint(low=0, high=k, size=len(df_fut)) #randomly identify indices
    R[:,i] = Er[tuple(np.arange(0,len(df_fut),1)),I] #sample from Er at random indices I
    
E = R #set E to equal R

/cluster/tufts/hpc/tools/anaconda/202105/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


One needs to experiment with different k values to find the appropriate k for each basin by going trough all the next steps of the SWM and diagnostic figures.  

### <font color='green'> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Step 2: Generate Stochastic Log Errors </font>

In [9]:
# l is a matrix of [len(data),m*n] representing stochastic log-ratio errors         
L = np.zeros((len(df_fut),m*n)) #initialize empty array to populate below

# for the first five time periods, we set values to be the deterministic model lambdas across all realizations
# this is because we do not have enough time periods in the past to populate the AR4 model stochastically
for i in range(0,m*n):
    L[0:5,i] = data['lambda'].iloc[0:5] 

for i in range(0,m):    
    B = Beta[i,:] #let B = random parameter set for realization i
    for t in range(5,len(df_fut)):
        # L(t) = B0 + B1*L[t-1] + B2*L[t-2] + B3*L[t-3] + B4*L[t-4] + E[t]
        L[t, i*n:((i+1)*n)] = B[0] + \
                              B[1] * L[t-1, i*n:((i+1)*n)] + \
                              B[2] * L[t-2, i*n:((i+1)*n)] + \
                              B[3] * L[t-3, i*n:((i+1)*n)] + \
                              B[4] * L[t-4, i*n:((i+1)*n)] + \
                              E[t, i*n:((i+1)*n)]

### <font color = 'green'> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Step 3: Generate Stochastic Streamflows </font>

In [10]:
# Q is a matrix of stochastically simulated streamflows of size [m*n, len(data)]
# Just like lambda = log(Qmodel/Qgage), we now set Q = Qmodel/e^L where
# L is stochastic lambdas and Q is stochastic streamflow
Q = np.array(df_fut['Qfut']) / np.exp(L.T)

# BCF is transformation bias correction factor 
# this address the bias introduced by moving to log space
BCF = 1 / np.exp(-np.mean(data['lambda']) + np.var(data['lambda'])/2)

Q = Q*BCF

In [11]:
print(BCF)

0.9807641078478005


In [12]:
display(pd.DataFrame(Q))

,0,1,2,3,4,5,6,7,8,9,...,11313,11314,11315,11316,11317,11318,11319,11320,11321,11322
0,13.338101,11.055095,14.752563,15.381338,24.03346,27.462582,37.066827,41.651897,49.336763,52.144416,...,83.726171,68.380558,66.163261,55.467380,79.033734,62.722628,75.124209,63.989184,61.190060,83.800474
1,13.338101,11.055095,14.752563,15.381338,24.03346,28.804395,30.151047,43.365044,42.229492,47.972931,...,71.949402,74.317626,69.025338,82.776072,67.388525,65.441663,86.032443,68.260365,45.651647,94.508607
2,13.338101,11.055095,14.752563,15.381338,24.03346,34.298920,36.533298,32.295000,45.681188,47.094399,...,64.400372,20.232613,46.645445,46.492153,54.659780,59.767849,69.972281,56.739515,91.347567,76.011918
3,13.338101,11.055095,14.752563,15.381338,24.03346,31.958939,25.292702,31.171070,46.684317,44.513730,...,64.267747,81.532979,72.055936,72.562654,67.271677,65.463198,64.107344,63.878799,83.292347,102.266901
4,13.338101,11.055095,14.752563,15.381338,24.03346,28.486155,9.608049,36.281380,59.858387,37.634954,...,92.034372,82.518599,109.895107,72.700481,127.779902,79.672847,80.507163,82.162250,86.028596,99.546658
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,13.338101,11.055095,14.752563,15.381338,24.03346,27.730236,34.397972,49.251683,40.523447,52.488078,...,75.692043,70.717252,94.397615,69.603722,107.314771,87.619938,86.287701,75.236170,89.693003,85.191230
9996,13.338101,11.055095,14.752563,15.381338,24.03346,35.488277,32.997268,42.819133,40.755395,47.900158,...,95.465242,127.551045,93.931779,122.562189,85.575967,80.752153,119.531689,77.127160,78.533151,146.128486
9997,13.338101,11.055095,14.752563,15.381338,24.03346,13.558002,23.162066,28.515688,36.702637,37.327778,...,114.139071,66.860362,67.912432,86.793129,101.250199,82.371991,74.206951,74.021439,92.709387,75.755005
9998,13.338101,11.055095,14.752563,15.381338,24.03346,41.955914,32.452618,42.820868,46.662777,15.812081,...,111.584557,96.145709,143.882801,96.956754,95.409915,80.342445,76.263440,92.191960,42.399738,77.031988


In [13]:
display(pd.DataFrame(L.T))

,0,1,2,3,4,5,6,7,8,9,...,11313,11314,11315,11316,11317,11318,11319,11320,11321,11322
0,1.810035,1.996299,1.706123,1.66287,1.214848,1.079892,0.778434,0.660735,0.490112,0.433090,...,-0.088721,0.109699,0.139391,0.312966,-0.044317,0.183025,-0.001438,0.155078,0.234308,0.156012
1,1.810035,1.996299,1.706123,1.66287,1.214848,1.032188,0.984937,0.620429,0.645663,0.516470,...,0.062868,0.026439,0.097043,-0.087378,0.115083,0.140588,-0.137020,0.090463,0.527253,0.035759
2,1.810035,1.996299,1.706123,1.66287,1.214848,0.857602,0.792933,0.915170,0.567095,0.534953,...,0.173711,1.327492,0.488941,0.489478,0.324430,0.231280,0.069606,0.275321,-0.166379,0.253560
3,1.810035,1.996299,1.706123,1.66287,1.214848,0.928264,1.160641,0.950592,0.545373,0.591309,...,0.175773,-0.066220,0.054074,0.044311,0.116819,0.140259,0.157146,0.156804,-0.074064,-0.043136
4,1.810035,1.996299,1.706123,1.66287,1.214848,1.043298,2.128556,0.798778,0.296800,0.759174,...,-0.183331,-0.078236,-0.368010,0.042413,-0.524751,-0.056181,-0.070641,-0.094904,-0.106387,-0.016176
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1.810035,1.996299,1.706123,1.66287,1.214848,1.070193,0.853159,0.493139,0.686901,0.426521,...,0.012158,0.076098,-0.216000,0.085943,-0.350208,-0.151261,-0.139982,-0.006840,-0.148100,0.139552
9996,1.810035,1.996299,1.706123,1.66287,1.214848,0.823514,0.894732,0.633097,0.681193,0.517988,...,-0.219932,-0.513729,-0.211053,-0.479857,-0.123847,-0.069637,-0.465877,-0.031664,-0.015229,-0.400036
9997,1.810035,1.996299,1.706123,1.66287,1.214848,1.785739,1.248641,1.039628,0.785933,0.767369,...,-0.398587,0.132181,0.113297,-0.134766,-0.292037,-0.089498,0.010847,0.009437,-0.181177,0.256946
9998,1.810035,1.996299,1.706123,1.66287,1.214848,0.656097,0.911376,0.633057,0.545835,1.626333,...,-0.375952,-0.231077,-0.637483,-0.245504,-0.232625,-0.064551,-0.016489,-0.210081,0.601150,0.240229


In [14]:
data

,date,Qgage,Qmodel,month,lambda,raw diff,Ar_res
0,1987-10-01,59.938142,366.261850,10,1.810035,306.323708,1.786596
1,1987-10-02,99.896904,735.416652,10,1.996299,635.519749,1.411201
2,1987-10-03,99.896904,550.188717,10,1.706123,450.291814,0.761985
3,1987-10-04,79.917523,421.519228,10,1.662870,341.601705,0.583921
4,1987-10-05,99.896904,336.630618,10,1.214848,236.733714,0.070037
...,...,...,...,...,...,...,...
11318,2018-09-26,79.917523,78.229589,9,-0.021347,-1.687934,-0.022115
11319,2018-09-27,79.917523,77.855393,9,-0.026142,-2.062129,-0.024599
11320,2018-09-28,59.938142,77.513991,9,0.257145,17.575849,0.261246
11321,2018-09-29,79.917523,81.235286,9,0.016355,1.317763,-0.024250


## <font color = 'green'> Save Outputs </font>

Outputs saved as **.npy** files. The outputs are:

1. E.npy: stochastic residuals
2. L.npy: stochastic log ratios
3. Q.npy: stochastic streamflows

In [15]:
# save the results as .npy files
np.save('Q_2degree.npy',Q)
np.save('StochasticE_2degree.npy',E)
np.save('L_2degree.npy',L)

***

# <center> <font color = 'red'> Part 2: Diagnostic Figures </font> </center>

Here, we include code to generate T-year flood flows for the specified warming scenario.

## <font color = 'green'> Reloading in the data </font>

In [16]:
# functions developed by Ghazal:
import Func_Lib_forAbby as func

# general packages
import numpy as np #numpy is a scientific computing package
import pandas as pd #pandas is a data analysis package
import math #math contains math functions
import time #functions in this package can be used to understand how long the code is taking to run
import os #functions related to operating system

# plotting packages
import matplotlib.pyplot as plt #matplotlib.pyplot is used for plotting
import seaborn as sns #seaborn is a plotting package
plt.style.use('seaborn-white') #this is the specific style of plotting used

# statistical analysis packages
from scipy import stats #scipy.stats are functions related to statistical analysis
from scipy.stats import norm #functions for normally distributed random variable 

# time series analysis packages
import statsmodels as sm
from statsmodels.graphics.tsaplots import plot_acf #function to plot autocorrelation function (acf) for time series analysis
from statsmodels.graphics.tsaplots import plot_pacf #function to plot partial autocorrelation function (pacf) for time series analysis
#from statsmodels.tsa.arima_model import ARIMA #import function for ARIMA (time series analysis)
from statsmodels.tsa.arima.model import ARIMA

In [17]:
# read in the csv of streamflow data and assign to variable called "data"
data = pd.read_csv('SWM-data_cond-debias.csv') #columns are date, Qgage (observed), Qmodel (deterministic model), and diff (Qmodel - Qgage)

data['month'] = pd.DatetimeIndex(data['date']).month #create a DatetimeIndex column for month
data['date'] = pd.to_datetime(data['date']) #convert format of "date" column to Datetime (for time series analysis)

#convert units to m^3/s 
# mm/day * SHA area in mi2 * (1609.34^2 m^2/mi^2) * (10^-3 m/1 mm) * (1 day/24*3600 s)
area_mi2_SHA = 6665 #area for Shasta baisn in sq. mi. from Zach Brodeur
data['Qgage'] = data['Qgage']*area_mi2_SHA*(1609.34**2)*(10**-3)/(24*3600)
data['Qmodel'] = data['Qmodel']*area_mi2_SHA*(1609.34**2)*(10**-3)/(24*3600)

# generating empirical log-ratio errors
data['lambda'] = np.log(data['Qmodel'] / data['Qgage']) #calculate the lambda = log(Qmodel/Qgage)
data['raw diff'] = data['Qmodel'] - data['Qgage']
data #display first five rows of data

,date,Qgage,Qmodel,month,lambda,raw diff
0,1987-10-01,59.938142,366.261850,10,1.810035,306.323708
1,1987-10-02,99.896904,735.416652,10,1.996299,635.519749
2,1987-10-03,99.896904,550.188717,10,1.706123,450.291814
3,1987-10-04,79.917523,421.519228,10,1.662870,341.601705
4,1987-10-05,99.896904,336.630618,10,1.214848,236.733714
...,...,...,...,...,...,...
11318,2018-09-26,79.917523,78.229589,9,-0.021347,-1.687934
11319,2018-09-27,79.917523,77.855393,9,-0.026142,-2.062129
11320,2018-09-28,59.938142,77.513991,9,0.257145,17.575849
11321,2018-09-29,79.917523,81.235286,9,0.016355,1.317763


In [18]:
#load in a "future" scenario into a new datafrae df_fut
df_fut = pd.read_csv('SWM-data_cond-debias_scenario-12.csv') #make sure this is one of the five we want, I just used a random one

#put the future scenario into the correct units of m3/s
df_fut['Qfut'] = df_fut['Qsim_cbias']*area_mi2_SHA*(1609.34**2)*(10**-3)/(24*3600)
df_fut['date'] = pd.to_datetime(df_fut['date'])
df_fut = df_fut[(df_fut['date'] > '1987-09-30') & (df_fut['date'] < '2018-10-01')]
df_fut = df_fut.reset_index()
df_fut['index']=np.arange(0, 11323)

#df_fut = df_fut.iloc[0:11323]

df_fut

,index,date,Qsim,Qsim_cbias,Qfut
0,0,1987-10-01,0.008558,0.415945,83.103222
1,1,1987-10-02,0.008422,0.415334,82.981132
2,2,1987-10-03,0.008271,0.414649,82.844289
3,3,1987-10-04,0.008132,0.414021,82.718931
4,4,1987-10-05,0.007975,0.413306,82.575901
...,...,...,...,...,...
11318,11318,2018-09-26,0.012123,0.384383,76.797403
11319,11319,2018-09-27,0.011791,0.382832,76.487560
11320,11320,2018-09-28,0.011474,0.381337,76.188862
11321,11321,2018-09-29,0.014408,0.394723,78.863275


In [19]:
Q = np.load('Q_2degree.npy')
m=100
n=100

In [20]:
pd.DataFrame(Q)

,0,1,2,3,4,5,6,7,8,9,...,11313,11314,11315,11316,11317,11318,11319,11320,11321,11322
0,13.338101,11.055095,14.752563,15.381338,24.03346,27.462582,37.066827,41.651897,49.336763,52.144416,...,83.726171,68.380558,66.163261,55.467380,79.033734,62.722628,75.124209,63.989184,61.190060,83.800474
1,13.338101,11.055095,14.752563,15.381338,24.03346,28.804395,30.151047,43.365044,42.229492,47.972931,...,71.949402,74.317626,69.025338,82.776072,67.388525,65.441663,86.032443,68.260365,45.651647,94.508607
2,13.338101,11.055095,14.752563,15.381338,24.03346,34.298920,36.533298,32.295000,45.681188,47.094399,...,64.400372,20.232613,46.645445,46.492153,54.659780,59.767849,69.972281,56.739515,91.347567,76.011918
3,13.338101,11.055095,14.752563,15.381338,24.03346,31.958939,25.292702,31.171070,46.684317,44.513730,...,64.267747,81.532979,72.055936,72.562654,67.271677,65.463198,64.107344,63.878799,83.292347,102.266901
4,13.338101,11.055095,14.752563,15.381338,24.03346,28.486155,9.608049,36.281380,59.858387,37.634954,...,92.034372,82.518599,109.895107,72.700481,127.779902,79.672847,80.507163,82.162250,86.028596,99.546658
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,13.338101,11.055095,14.752563,15.381338,24.03346,27.730236,34.397972,49.251683,40.523447,52.488078,...,75.692043,70.717252,94.397615,69.603722,107.314771,87.619938,86.287701,75.236170,89.693003,85.191230
9996,13.338101,11.055095,14.752563,15.381338,24.03346,35.488277,32.997268,42.819133,40.755395,47.900158,...,95.465242,127.551045,93.931779,122.562189,85.575967,80.752153,119.531689,77.127160,78.533151,146.128486
9997,13.338101,11.055095,14.752563,15.381338,24.03346,13.558002,23.162066,28.515688,36.702637,37.327778,...,114.139071,66.860362,67.912432,86.793129,101.250199,82.371991,74.206951,74.021439,92.709387,75.755005
9998,13.338101,11.055095,14.752563,15.381338,24.03346,41.955914,32.452618,42.820868,46.662777,15.812081,...,111.584557,96.145709,143.882801,96.956754,95.409915,80.342445,76.263440,92.191960,42.399738,77.031988


## <font color = 'green'> Generating Figure Inputs </font>

In [21]:
def annualmax(Q,data,n):
    annual_max=np.array(pd.DataFrame(Q.T,index=data['date']).resample('Y').max())
    annualmaxsort = np.zeros(annual_max.shape)
    for i in range(0,n):
        annualmaxsort[:,i]=np.sort(annual_max[:,i])[::-1]
    
    annualmaxsort = pd.DataFrame(annualmaxsort)
    #annualmaxsort['rank']=(annualmaxsort.index) + 1
    #annualmaxsort['Exeedance']=annualmaxsort['rank']/(len(annualmaxsort)+1)
    #annualmaxsort['non_Exceedance']=1-annualmaxsort['Exeedance']
    #Data=data.set_index('date')
    #annualmaxobs=Data['Qgage'].resample('Y').max()
    #annualmaxmodel=Data['Qmodel'].resample('Y').max()
    #annualmaxsort['obs']=np.sort(annualmaxobs)[::-1]
    #annualmaxsort['model']=np.sort(annualmaxmodel)[::-1]
    amax=np.array(annualmaxsort)[:,0:n]
    return annualmaxsort
Annualmax = annualmax(Q, df_fut, m*n)

In [22]:
# calculating 50-year floods
from scipy.stats import pearson3
ann_max=np.array(Annualmax)
P=1-(1/50)
flood50Y2=np.exp(np.log(pd.DataFrame(ann_max[:,0:10000])).mean()+np.log(pd.DataFrame((ann_max[:,0:10000]))).std()*abs(pearson3.ppf(P, np.log(pd.DataFrame(ann_max[:,0:10000])).skew())))
flood50Y2 = np.array(flood50Y2)

In [23]:
# calculating 100-year floods
from scipy.stats import pearson3
ann_max=np.array(Annualmax)
P=1-(1/100)
flood100Y2=np.exp(np.log(pd.DataFrame(ann_max[:,0:10000])).mean()+np.log(pd.DataFrame((ann_max[:,0:10000]))).std()*abs(pearson3.ppf(P, np.log(pd.DataFrame(ann_max[:,0:10000])).skew())))
flood100Y2 = np.array(flood100Y2)

In [24]:
# calculating 500-year floods
from scipy.stats import pearson3
ann_max=np.array(Annualmax)
P=1-(1/500)
flood500Y2=np.exp(np.log(pd.DataFrame(ann_max[:,0:10000])).mean()+np.log(pd.DataFrame((ann_max[:,0:10000]))).std()*abs(pearson3.ppf(P, np.log(pd.DataFrame(ann_max[:,0:10000])).skew())))
flood500Y2 = np.array(flood500Y2)

In [25]:
np.save('flood50Y2.npy',flood50Y2)
np.save('flood100Y2.npy',flood100Y2)
np.save('flood500Y2.npy',flood500Y2)